In [1]:
from pyspark.sql import SparkSession

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-complete") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-complete").getOrCreate()
spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/02 16:32:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/02 16:32:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse


line0 = '_:nb53a244 <http://schema.org/value1> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line1 = '_:nb53a244 <http://schema.org/value2> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line2 = '_:different <http://schema.org/value3> "zfgrredzfzef" <https://youtube.com>   .'
line3 = '_:nb53a244 <http://schema.org/value4> "dqqsd" <https://different.com>   .'

line4 = '<https://google.com> <http://schema.org/value4> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line5 = '<https://google.com> <http://schema.org/value4> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line6 = '<https://different.com> <http://schema.org/value6> "fsqfqsfdqs" <https://youtube.com>   .'
line7 = '<https://google.com> <http://schema.org/type> "Recipe" <https://different.com>   .'

line8 = '<https://google.com> <http://schema.org/type> "Recipe"'
line9 = '<https://google.com> <http://schema.org/type>'
line10 = '<https://google.com>'
line11 = ''

quad_motif = re.compile(r'([^\s]+)\s([^\s]+)\s(.+)\s([^\s]+)\s+\.')

def parseQ(l, parts):
    result = quad_motif.match(l)
    if result:
        sub = result.group(1).strip()
        
        pred = result.group(2).strip()
        pred = re.sub(r'([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?', '', pred)
        if pred == "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
            pred = "isa:" + re.sub(r'([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?', '', result.group(3).strip())

        if sub.startswith("_:"):
            sub += result.group(4).strip()
        return Row(subject=sub, predicate=pred, hashdom=hash(sub) % parts)
    else:
        # Handle non-matching lines
        print(f"parsing error : {l}")
        return None

print(parseQ(line0, 100))
print(parseQ(line1, 100))
print(parseQ(line2, 100))
print(parseQ(line3, 100))
print(parseQ(line4, 100))
print(parseQ(line5, 100))
print(parseQ(line6, 100))
print(parseQ(line7, 100))
print(parseQ(line8, 100))
print(parseQ(line9, 100))
print(parseQ(line10, 100))
print(parseQ(line11, 100))

Row(subject='_:nb53a244<https://youtube.com>', predicate='<schema.org/value1>', hashdom=38)
Row(subject='_:nb53a244<https://youtube.com>', predicate='<schema.org/value2>', hashdom=38)
Row(subject='_:different<https://youtube.com>', predicate='<schema.org/value3>', hashdom=83)
Row(subject='_:nb53a244<https://different.com>', predicate='<schema.org/value4>', hashdom=39)
Row(subject='<https://google.com>', predicate='<schema.org/value4>', hashdom=19)
Row(subject='<https://google.com>', predicate='<schema.org/value4>', hashdom=19)
Row(subject='<https://different.com>', predicate='<schema.org/value6>', hashdom=5)
Row(subject='<https://google.com>', predicate='<schema.org/type>', hashdom=19)
parsing error : <https://google.com> <http://schema.org/type> "Recipe"
None
parsing error : <https://google.com> <http://schema.org/type>
None
parsing error : <https://google.com>
None
parsing error : 
None


In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import random 

# pip install tldextract pas internet
import tldextract

@udf('string')
def extract_tld(url):
    tld = tldextract.extract(url)
    if tld.registered_domain:
        return tld.registered_domain
    else:
        # in case the URL is an IP
        return tld.domain

spark.udf.register("extract_tld", extract_tld)

# TODO : faire la fonction sample_tab
# Note : le "@udf('string')" permet de faire une fonction s'executant en pyspark.sql

@udf(ArrayType(StringType()))
def sample_tab(input_array):
    # Vérifiez si l'entrée est une liste non vide
    if input_array and isinstance(input_array, list):
        # Utilisez random.sample pour sélectionner aléatoirement 10 éléments sans remplacement
        selected_elements = random.sample(input_array, min(10, len(input_array)))
        return selected_elements
    else:
        return None  # Retournez None si l'entrée n'est pas valide

spark.udf.register("sample_tab", sample_tab)

from pyspark.sql.functions import col
def partCS(hashdom_val, hashdom_max, output=None):
    print(f"part {hashdom_val+1}/{hashdom_max} started");
    file_name = f"hashdom{hashdom_val+1}-{hashdom_max}"
    

    # calcule aussi les domaines et sous-domaines distincts et leur nombre
    # TODO : faire fonction prenant en paramètre le resultat de COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')) as urls 
    # et qui retourne 10 éléments pris au hasard (le sample)
    result = spark.sql(f"""
        SELECT
            pset,
            COUNT(subject) as count,
            CONCAT_WS(' ', COLLECT_SET(extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')))) as distinct_domains,
            CONCAT_WS(' ', sample_tab(COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')))) as urls
        FROM (
            SELECT
                subject,
                CONCAT_WS(' ', SORT_ARRAY(COLLECT_SET(predicate))) as pset
            FROM
                Super
            WHERE
                hashdom={hashdom_val}
            GROUP BY
                subject
        )
        GROUP BY
            pset
    """).cache()
    
    # result = result.withColumn("tld", tld_extract_udf(col("subject")))
    
    result.show(truncate=0)

    
    if(output is not None):
        print("Saving")
        result.write.option("header",True) \
            .mode("overwrite") \
            .csv(f"s3a://test-out/{output}/{file_name}")
    
    # clear variables from memory
    #cset.unpersist()
    result.unpersist()

    print(f"part {hashdom_val+1}/{hashdom_max} finished");
    
    del result

def completeCS(input, parts, output=None):
    lines = spark.sparkContext.textFile(f"s3a://{input}/**")
    
    if(lines.isEmpty()):
       print(f"error: {input} folder empty")
    
    first_10_lines = lines.take(10)
    for line in first_10_lines:
        print(line)
    
    parsedLines = lines.map(lambda l: parseQ(l, parts)).filter(lambda x: x is not None).toDF()
    parsedLines.show(truncate=0)

    parsedLines.createOrReplaceTempView("Super")
    
    for i in range(parts):
        partCS(i, parts, output)
        
    print("Finished")

In [4]:
#print(spark.sql("""select * from """).show())

In [ ]:
# input : test or wdc or wdc-2023
# no output = no save
completeCS("wdc", 20, "test/wdc-with-domains")

23/12/02 16:32:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


<http://1000lifehacks.com/#organization> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Organization> <http://1000lifehacks.com/how-to-remember-stuff-in-the-morning/>   .
<http://1000lifehacks.com/#organization> <http://schema.org/image> <http://1000lifehacks.com/#logo> <http://1000lifehacks.com/how-to-remember-stuff-in-the-morning/>   .
<http://1000lifehacks.com/#organization> <http://schema.org/logo> <http://1000lifehacks.com/#logo> <http://1000lifehacks.com/how-to-remember-stuff-in-the-morning/>   .
<http://1000lifehacks.com/#organization> <http://schema.org/name> "1000 Life Hacks" <http://1000lifehacks.com/how-to-remember-stuff-in-the-morning/>   .
<http://1000lifehacks.com/#organization> <http://schema.org/sameAs> <http://facebook.com/1000lifehacks> <http://1000lifehacks.com/how-to-remember-stuff-in-the-morning/>   .
<http://1000lifehacks.com/#organization> <http://schema.org/sameAs> <http://instagram.com/lifehack.ers> <http://1000lifehacks.com/how-to-remembe

+----------------------------------------+--------------------------------------+-------+
|subject                                 |predicate                             |hashdom|
+----------------------------------------+--------------------------------------+-------+
|<http://1000lifehacks.com/#organization>|<w3.org/1999/02/22-rdf-syntax-ns#type>|16     |
|<http://1000lifehacks.com/#organization>|<schema.org/image>                    |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/logo>                     |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/name>                     |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/sameAs>                   |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/sameAs>                   |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/sameAs>                   |16     |
|<http://1000lifehacks.com/#organization>|<schema.org/url>                      |16     |
|<http://1